In [12]:
model="gpt-35-turbo"
import os
openai_api_base = os.environ["OPENAI_API_BASE_URL"];

# 回顾

In [13]:
from langchain.chains import LLMRequestsChain
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name=model, temperature=0,openai_api_base= openai_api_base,max_tokens=200)
template = """Between >>> and <<< are the raw search result text from web.
Extract the answer to the question '{query}' or say "not found" if the information is not contained.
Use the format
Extracted:<answer or "not found">
>>> {requests_result} <<<
Extracted:"""

PROMPT = PromptTemplate(
  input_variables=["query", "requests_result"],
  template=template,
)


query_chain = LLMRequestsChain(llm_chain = LLMChain(llm=llm, prompt=PROMPT), output_key="query_info")


In [14]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.chains import SequentialChain

translating_prompt_template = """
translate "{query_info}" into English:

"""

translating_chain = LLMChain(
    llm = llm,
    prompt=PromptTemplate.from_template(translating_prompt_template),
    output_key = "translated"
)




In [15]:
llm = ChatOpenAI(model_name=model, temperature=0, max_tokens=200,openai_api_base= openai_api_base)

def overall(question):
    inputs = {
      "query": question,
      "url": "http://www.baidu.com/s?wd=" + question.replace(" ", "+")
      # "url": "https://cn.bing.com/search?q=" + question.replace(" ", "+")
    }
    
    overall_chain = SequentialChain(
        chains=[query_chain, translating_chain],
        input_variables=["query","url"],
        output_variables=["translated"],
        verbose=True
    )
    
    return overall_chain(inputs)["translated"]

overall("北京今天天气")



> Entering new SequentialChain chain...

> Finished chain.


'"Extracted: Strong wind blue warning issued at 16:35 on the 15th. The temperature is 8°C with overcast skies and west wind of level 1. The range of temperature is 2°C to 9°C. The air quality is rated as good. UV index levels for the given time are as follows: 19:00 - 2, 20:00 - 2, 21:00 - 2, 22:00 - 2, 23:00 - 2, 01:00 - 1, 02:00 - 2. The temperature levels for the given time are as follows: 19:00 - 8°C, 20:00 - 6°C, 21:00 - 6°C, 22:00 - 5°C, 23:00 - 4°C, 01:00 - 4°C, 02:00 - 3°C."'

# 没有记忆的模型

In [16]:
from openai import OpenAI
client = OpenAI(
    base_url = openai_api_base
)
def get_response(input):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are an AI assistant."},   
            {"role": "user", "content": input}
        ],
        temperature = 0.9, 
        max_tokens = 200
      )
    return response.choices[0].message.content

In [17]:
print(get_response("你好，今天是周二我要去健身，我一般每周二健身。你今天干什么？"))
print(get_response("我一般周几健身？"))

你好！作为一个AI助手，我没有身体，所以不需要健身。我会继续帮助你解答问题和提供信息。有什么我可以帮到你的吗？
作为一个AI助手，我无法准确知道您的个人日常安排。通常，健身时间是根据个人偏好和日程安排来确定的。大多数人选择在工作日或周末进行健身训练。您可以根据自己的时间表和喜好选择一周中适合您的时间进行健身。重要的是保持一致性并将健身活动纳入您的日常生活中。


# 通过Gradio快速展示功能

In [18]:
!pip3 install gradio

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [20]:
import gradio as gr
def respond(message, chat_history):
        bot_message = get_response(message)
        chat_history.append((message, bot_message)) #保存历史对话记录，用于显示
        return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) #对话框
    msg = gr.Textbox(label="Prompt") #输入框
    btn = gr.Button("Submit") #提交按钮
    #提交
    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) 
gr.close_all()
demo.launch(share=True)

/Users/wjx/Library/Python/3.9/lib/python/site-packages/gradio/blocks.py:928: UserWarning: api_name respond already exists, using respond_1
  warnings.warn(f"api_name {api_name} already exists, using {api_name_}")


Running on local URL:  http://127.0.0.1:7867
Running on public URL: https://7ffaf96041cb63ecdd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# 提供外部记忆

In [21]:
import openai
def get_response(msg):
    # print(msg)
    response = client.chat.completions.create(
        model=model, 
        messages=msg,
        temperature = 0.9, 
        max_tokens = 600
    )
    return response.choices[0].message.content

In [22]:

def history_to_prompt(chat_history): # 将对话内容保存在一个List里
    msg = [{"role": "system", "content": "You are an AI assistant."}]
    i = 0
    for round_trip in chat_history: # 将List里的内容，组成 ChatCompletion的 messages部分，{role，content} dict
        msg.append({"role": "user", "content": round_trip[0]})
        msg.append({"role": "assistant", "content": round_trip[1]})
    return msg

def respond(message, chat_history):
    his_msg = history_to_prompt(chat_history) #并装历史会话，ChatCompletion的 messages部分格式
    his_msg.append({"role": "user", "content": message}) # 放入当前用户问题
    bot_message = get_response(his_msg)
    chat_history.append((message, bot_message)) # 将用户问题和返回保存到 历史记录 List
    return "", chat_history



In [23]:
import gradio as gr
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=480) #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit
gr.close_all()
demo.launch(share=True)

/Users/wjx/Library/Python/3.9/lib/python/site-packages/gradio/blocks.py:928: UserWarning: api_name respond already exists, using respond_1
  warnings.warn(f"api_name {api_name} already exists, using {api_name_}")


Running on local URL:  http://127.0.0.1:7868
Running on public URL: https://71255d3bb2758b6abc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
